In [40]:
import numpy as np
import pandas as pd 

После загрузги библиотеки я сразу посмотрела, какие столбы загружаются, и оставила нужные, дабы при дальнейших загрузках на это не обращать внимание. 

In [23]:
data_xl = pd.ExcelFile("test_task.xlsx")
data = data_xl.parse(data_xl.sheet_names[0])
data.drop('good (1)', axis= 1 , inplace= True )
data.head()

,area,cluster,cluster_name,keyword,count,x,y
0,eligibility,0.0,Кластер 0,several animated buried,1260,5.772342,12.564796257345005
1,eligibility,0.0,Кластер 0,singles unusual buyers,866,14.829280,7.8507285727125815
2,eligibility,0.0,Кластер 0,hawaiian directive,163,11.381856,3.8981370219558604
3,eligibility,0.0,Кластер 0,dynamics directly,1146,9.980149,6.281427914064545
4,eligibility,1.0,Кластер 1,decision surgeons montreal,823,3.283940,4.39674063521296


Прежде чем что-то делать, надо узнать есть ли пропуски. 

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          228 non-null    object 
 1   cluster       228 non-null    float64
 2   cluster_name  228 non-null    object 
 3   keyword       228 non-null    object 
 4   count         227 non-null    object 
 5   x             228 non-null    float64
 6   y             228 non-null    object 
dtypes: float64(2), object(5)
memory usage: 12.6+ KB


In [25]:
data.isnull().sum()[lambda x: x>0]

area            1
cluster         1
cluster_name    1
keyword         1
count           2
x               1
y               1
dtype: int64

Выяснилось, что они есть. Но в странном соотношении. Надо бы посмотреть на эти пропуски. 

In [28]:
list_nan= data.index[data.isnull().any(axis=1)]
data.iloc[list_nan]

,area,cluster,cluster_name,keyword,count,x,y
178,housewives,2.0,Кластер 2,outstanding relations,NaN,1.289158,6.397514584626231
193,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Это оказалась целиком пустая строку, которую мы просто удалим. А вот с единичным значениме в инструкции нет ничего. Так бы по хорошему надо было б узнать, можно ли его заменить средним значением. Т.к. предполагаем, что тестовое мы делаем без вопросов, то эту строчку мы то же удаляем. 

In [29]:
data = data.drop(index = list_nan)

In [30]:
data.isnull().sum()[lambda x: x>0]

Series([], dtype: int64)

Все, пропусков у нас нет. 
В условии у нас указан порядок: area, cluster, cluster_name, keyword, x, y, count, color.

Значит перемещаем столбец count в конец. Так же создаем и заполняем столбец color по правилам, которые есть в задании. 

In [32]:
data = data[['area', 'cluster', 'cluster_name', 'keyword', 'x', 'y', 'count']]
data.head()

,area,cluster,cluster_name,keyword,x,y,count
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796257345005,1260
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.8507285727125815,866
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.8981370219558604,163
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281427914064545,1146
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.39674063521296,823


In [34]:
data['color'] = pd.Series()
data.head()

C:\Users\juiwa\AppData\Local\Temp\ipykernel_5368\1155315422.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['color'] = pd.Series()


,area,cluster,cluster_name,keyword,x,y,count,color
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796257345005,1260,NaN
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.8507285727125815,866,NaN
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.8981370219558604,163,NaN
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281427914064545,1146,NaN
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.39674063521296,823,NaN


Цвет задается каждому словосочетанию согласно следующими правилам:
* внутри одной области цвета словосочетаний в одном кластере должны быть одинаковые, в разных - отличаться (например, у "Кластер 1" все слова будут окрашены в красный, у "Кластер 2" - в зеленый и т.д.)
* цвета кластеров в разных областях могут повторяться
* цвета кластеров в разных областях с разным номером не имеют никакой связи (у одной области [area] слова из "Кластер 1" могут быть красного цвета, в другой области у слов из "Кластер 1" может быть другой цвет)

По данному условию мы можем найти какое количество обозначений кластеров у нас есть - это и будет колличество наших цветов.  


In [36]:
data['cluster'].value_counts()

1.0    62
0.0    60
2.0    60
3.0    45
Name: cluster, dtype: int64

Значит будет 4 цвета. Т.к. их мало и не было сказано, как именно их надо задавать, то я задам их в ручную. В ином случае разумно былоб сделать генерацию цвета с привязкой к номеру. Т.е. цвета обозначать номерами. 

In [37]:
dict_color = {0: 'red',
              1:'blue',
              2: 'green',
              3: 'yellow'}

In [41]:
for key, value in dict_color.items():
    data['color'] = np.where((data.cluster == key ), value, data.color)

In [42]:
data.head()

,area,cluster,cluster_name,keyword,x,y,count,color
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796257345005,1260,red
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.8507285727125815,866,red
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.8981370219558604,163,red
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281427914064545,1146,red
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.39674063521296,823,blue


AttributeError: 'Series' object has no attribute 'value'